# 🌟 XL 訓練器 by Hollowstrawberry

❗ **推薦使用 Colab Premium** 理想情況下，您可以將運行時間變更為 A100 並使用最大批次大小。
但是，如果您載入擴散器模型，您仍然可以免費訓練，只是需要更長的時間。


基於 [Kohya-ss](https://github.com/kohya-ss/sd-scripts) 與 [Linaqruf](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-dreambooth.ipynb) 來完成此工具，感謝。

### ⭕ 免則聲明
本文件是用於研究機器學習等前端技術為目的。
請閱讀以下文件說明 [Google Colab guidelines](https://research.google.com/colaboratory/faq.html) 與 [Terms of Service](https://research.google.com/colaboratory/tos_v3.html).

| |GitHub|🇬🇧 English|🇪🇸 Spanish|🇹🇼 繁體中文|
|:--|:-:|:-:|:-:|:-:|
| 🏠 **首頁** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab) | | |
| 📊 **資料集製作器** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Dataset_Maker.ipynb) | |
| ⭐ **Lora 訓練器** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Spanish_Lora_Trainer.ipynb) | [![在 Colab 開啟](https://raw.githubusercontent.com/hinablue/kohya-colab/main/assets/colab-badge-tw.svg)](https://colab.research.google.com/github/hinablue/kohya-colab/blob/main/Traditional_Chinese_Lora_Trainer.ipynb) |
| ⭐ **XL Lora 訓練器** | [![GitHub](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/github.svg)](https://github.com/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) | [![在 Colab 開啟](https://raw.githubusercontent.com/hinablue/kohya-colab/main/assets/colab-badge-tw.svg)](https://colab.research.google.com/github/hinablue/kohya-colab/blob/main/Traditional_Chinese_Lora_Trainer_XL.ipynb) |

In [ ]:
import os
import re
import toml
from time import time
from huggingface_hub import HfFileSystem
from IPython.display import Markdown, display

# These carry information from past executions
if "model_url" in globals():
  old_model_url = model_url
else:
  old_model_url = None
if "dependencies_installed" not in globals():
  dependencies_installed = False
if "model_file" not in globals():
  model_file = None

# These may be set by other cells, some are legacy
if "custom_dataset" not in globals():
  custom_dataset = None
if "override_dataset_config_file" not in globals():
  override_dataset_config_file = None
if "override_config_file" not in globals():
  override_config_file = None
if "continue_from_lora" not in globals():
  continue_from_lora = ""

COLAB = True
SOURCE = "https://github.com/qaneel/kohya-trainer"
COMMIT = None
BETTER_EPOCH_NAMES = True
LOAD_TRUNCATED_IMAGES = True
try:
  LOWRAM = int(next(line.split()[1] for line in open('/proc/meminfo') if "MemTotal" in line)) / (1024**2) < 15
except:
  LOWRAM = False

#@title ## 🚩 Start Here

#@markdown ### ▶️ 設定
#@markdown 你的專案名稱必須和包含圖片的資料夾名稱相同。不允許使用空格。
project_name = "" #@param {type:"string"}
#@markdown 資料夾結構不重要，只是為了方便。請確保每次都選擇相同的結構。我傾向使用以專案模式的方式。
folder_structure = "Organize by project (MyDrive/Loras/project_name/dataset)" #@param ["Organize by category (MyDrive/lora_training/datasets/project_name)", "Organize by project (MyDrive/Loras/project_name/dataset)"]
#@markdown 選擇並下載訓練所需要使用的模型。這些選項應該會產生乾淨且一致的結果。你也可以選擇自己的模型，只要貼上下載連結即可。
training_model = "Pony Diffusion V6 XL" #@param ["Pony Diffusion V6 XL", "Animagine XL V3", "Stable Diffusion XL 1.0 base"]
optional_custom_training_model_url = "" #@param {type:"string"}
#@markdown 自訂模型的下載連結。如果你沒有自訂模型，請留空。
load_diffusers = True #@param {type:"boolean"}

if optional_custom_training_model_url:
  model_url = optional_custom_training_model_url
elif "Pony" in training_model:
  if load_diffusers:
    model_url = "https://huggingface.co/stablediffusionapi/pony-diffusion-v6-xl/"
  else:
    model_url = "https://civitai.com/api/download/models/290640"
  model_file = "/content/ponyDiffusionV6XL.safetensors"
elif "Animagine" in training_model:
  if load_diffusers:
    model_url = "https://huggingface.co/cagliostrolab/animagine-xl-3.0"
  else:
    model_url = "https://civitai.com/api/download/models/293564"
  model_file = "/content/animagineXLV3.safetensors"
else:
  if load_diffusers:
    model_url = "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/"
  else:
    model_url = "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors"

if load_diffusers:
  vae_file= "stabilityai/sdxl-vae"
else:
  vae_url = "https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors"
  vae_file = "/content/sdxl_vae.safetensors"


#@markdown ### ▶️ 加工處理
resolution = 1024 #param {type:"slider", min:768, max:1536, step:128}
caption_extension = ".txt" #param {type:"string"}
#@markdown 這個選項會在訓練時自動翻轉圖片，不會增加訓練時間，但是可以讓模型學習更多。如果你的圖片少於 20 張，請務必開啟此選項。
#@markdown **如果你在意圖片的對稱性，請關閉此選項。**
flip_aug = False #@param {type:"boolean"}
#@markdown 針對動畫標籤進行洗牌可以提高學習和提示的效果。文字檔案開頭的啟動標籤，不會被洗牌。
shuffle_tags = True #@param {type:"boolean"}
shuffle_caption = shuffle_tags
#@markdown 啟動標籤數量。如果你的圖片沒有啟動標籤，請設定為 0。請務必確認你的啟動標籤放在文字檔案的最前面。
activation_tags = "1" #@param [0,1,2,3]
keep_tokens = int(activation_tags)

#@markdown ### ▶️ 步數
#@markdown 你的圖片會在訓練時重複這個次數。我建議你的圖片乘以重複次數介於 200 到 400 之間。
num_repeats = 2 #@param {type:"number"}
#@markdown 選擇你想要訓練的時間。一個好的起點是 10 個輪次（Epoch） 或 2000 個步數（Steps)。
#@markdown 每一個輪次（Epoch）的步數等於：你的圖片數量乘以重複次數，除以批次大小。
preferred_unit = "Epochs" #@param ["Epochs", "Steps"]
how_many = 10 #@param {type:"number"}
max_train_epochs = how_many if preferred_unit == "Epochs" else None
max_train_steps = how_many if preferred_unit == "Steps" else None
#@markdown 儲存每次的輪次（Epochs）可以讓你更好的比較你的 Lora 的進度。
save_every_n_epochs = 1 #@param {type:"number"}
keep_only_last_n_epochs = 10 #@param {type:"number"}
if not save_every_n_epochs:
  save_every_n_epochs = max_train_epochs
if not keep_only_last_n_epochs:
  keep_only_last_n_epochs = max_train_epochs

#@markdown ### ▶️ 學習設定
#@markdown 學習率是你的結果最重要的因素。如果你想要訓練更慢，或是你的圖片數量很多，或是你的 dim 和 alpha 很高，請把 UNet 的學習率調到 2e-4 或更低。
#@markdown 文字編碼器可以讓你的 Lora 學習概念更好。建議你把文字編碼器的學習率設定為 UNet 的一半或五分之一。如果你在訓練風格，你甚至可以把它設定為 0。
unet_lr = 3e-4 #@param {type:"number"}
text_encoder_lr = 6e-5 #@param {type:"number"}
#@markdown 調度器是指導學習率的演算法。如果你不確定，請選擇 `constant` 並忽略重啟數字（`lr_scheduler_number`）。若使用 `cosine_with_restarts` ，我個人建議使用 3 次重啟數字。
lr_scheduler = "cosine_with_restarts" #@param ["constant", "cosine", "cosine_with_restarts", "constant_with_warmup", "linear", "polynomial"]
lr_scheduler_number = 3 #@param {type:"number"}
lr_scheduler_num_cycles = lr_scheduler_number if lr_scheduler == "cosine_with_restarts" else 0
lr_scheduler_power = lr_scheduler_number if lr_scheduler == "polynomial" else 0
#@markdown 在訓練時，用來「熱身」學習率的步數（僅適用於 `constant_with_warmup`）。我建議你把它設定為 5%。
lr_warmup_ratio = 0.05 #@param {type:"slider", min:0.0, max:0.2, step:0.01}
lr_warmup_steps = 0
#@markdown 新功能，可以隨著時間調整損失函數，讓學習更有效率，並且可以用大約一半的輪次（Epochs）完成訓練。參考[論文](https://arxiv.org/abs/2303.09556)建議，使用 5% 的數值。
min_snr_gamma = 5.0 #@param {type:"slider", min:0.0, max:16.0, step:0.5}

#@markdown ### ▶️ 模型結構
#@markdown LoRA 是經典的類型，而 LoCon 則是適合風格類型的訓練。在 WebUI 中使用 LyCORIS 需要[這個擴充功能](https://github.com/KohakuBlueleaf/a1111-sd-webui-lycoris)。若想要更多資訊，請參考[這裡](https://github.com/KohakuBlueleaf/Lycoris)。
lora_type = "LoRA" #@param ["LoRA", "LoCon"]

#@markdown Below are some recommended XL values for the following settings:

#@markdown | type | network_dim | network_alpha | conv_dim | conv_alpha |
#@markdown | :---: | :---: | :---: | :---: | :---: |
#@markdown | Regular LoRA | 8 | 4 |   |   |
#@markdown | Style LoCon | 16 | 8 | 16 | 8 |

#@markdown 更大的 dim 代表更大的 Lora，它可以儲存更多資訊，但是並不是越大越好。建議的 dim 為 8-32，alpha 則是 dim 的一半。
network_dim = 8 #@param {type:"slider", min:1, max:32, step:1}
network_alpha = 4 #@param {type:"slider", min:1, max:32, step:1}
#@markdown 以下的數值不會影響 LoRA。它們的作用類似於 dim/alpha，但僅適用於 LyCORIS 的額外學習層。
conv_dim = 4 #@param {type:"slider", min:1, max:32, step:1}
conv_alpha = 1 #@param {type:"slider", min:1, max:32, step:1}
conv_compression = False #@param {type:"boolean"}

network_module = "lycoris.kohya" if "LoCon" in lora_type else "networks.lora"
network_args = None
if lora_type.lower() == "locon":
  network_args = [f"conv_dim={conv_dim}", f"conv_alpha={conv_alpha}"]

#@markdown ### ▶️ 更多設定
#@markdown 建議使用 0.15 ~ 0.4 之間的數值，數值越大，訓練生成的結果越接近正規化圖片。
prior_loss_weight = 0.1 #@param {type:"number"}
#@markdown 噪訊偏移可以改善亮度/暗度的處理結果。
noise_offset = 0.1 #@param {type:"number"}
#@markdown 黑魔法，建議值使用 2，若訓練真實人物等模型，可以使用 1。
clip_skip = 2 #@param {type:"slider", min:0, max:10, step:1}
#@markdown 最大文字標籤長度，預設為 225。
max_token_length = 225 #@param [75,125,225]
#@markdown 啟用不放大批次解析度，如果你想要使用 最大/最小 批次解析度，請設定為 `False`。
bucket_no_upscale = True #@param {type:"boolean"}

#@markdown **以下設定僅適用於不放大批次解析度為 `False` 時。**

#@markdown 若解析度小於 512，建議使用 256，否則建議使用 320。
min_bucket_reso = 256 #@param {type:"number"}
#@markdown 若解析度小於 512，建議使用 1024，否則建議使用 1280。
max_bucket_reso = 1024 #@param {type:"number"}

#@markdown 顏色增強可以改善結果的顏色。如果啟用，將會強制關閉快取潛在變數快取（`Latent Cache = False`）。
color_aug = False #@param {type:"boolean"}
#@markdown 依據臉部中心尺寸擷取圖片，再依此上限、下限決定擷取的範圍。可優化臉部訓練。若訓練包含背景風格可將此數值上限、下限加大。
face_crop_aug_range = [1.0, 3.0] #@param {type:"raw"}
#@markdown Random crop the image, recommend using False.
#@markdown 隨機擷取圖片區域，建議使用 `False`，若訓練風格類型則建議開啟。
random_crop = False #@param {type:"boolean"}
#@markdown 增加梯度累積步驟以節省 GPU 記憶體。但會降低訓練速度，並需要更高的學習率。
gradient_accumulation_steps = 1 #@param {type:"number"}

#@markdown ### ▶️ 實驗性功能
#@markdown 儲存額外資料，約 1 GB，可以讓你稍後繼續訓練。
save_state = False #@param {type:"boolean"}
#@markdown 如果有儲存的額外資料，則繼續訓練。
resume = False #@param {type:"boolean"}

#@markdown ### ▶️ 訓練設定
#@markdown 根據您的 Colab 配置調整這些參數。
#@markdown 如果您使用免費方案，則應在此儲存格頂部選擇模型。
#@markdown
#@markdown 批次大小越大通常速度越快，但會佔用更多記憶體。
train_batch_size = 4 #@param {type:"slider", min:1, max:16, step:1}
#@markdown 我發現 sdpa 和 xformers 之間沒有實質差異。
cross_attention = "sdpa" #@param ["sdpa", "xformers"]
#@markdown 如果您使用的是 A100，則應啟用 bf16。
mixed_precision = "fp16" #@param ["bf16", "fp16"]
#@markdown 快取潛在快取緩存，將在每個圖像旁邊添加一個 250KB 文件，但會使用相當少的記憶體。
cache_latents = True #@param {type:"boolean"}
cache_latents_to_drive = True #@param {type:"boolean"}
#@markdown 以下選項將關閉 shuffle_tags 並停用文字編碼器訓練。
cache_text_encoder_outputs  = False  # @param {type:"boolean"}

#@markdown ### ▶️ 進階設定
#@markdown 優化器是用於訓練的演算法。 AdanW8Bit 是預設設定並且效果很好，而 Prodigy 會自動管理學習率，並且可能具有多種優勢，例如由於需要更少的步驟而訓練速度更快，以及對於小型資料集更好地工作。
optimizer = "AdamW8bit" #@param ["AdamW8bit", "Prodigy", "DAdaptation", "DadaptAdam", "DadaptLion", "AdamW", "Lion", "SGDNesterov", "SGDNesterov8bit", "AdaFactor"]
#@markdown AdamW8bit 推薦使用: `weight_decay=0.1 betas=[0.9,0.99]`<p>
#@markdown Prodigy 推薦使用: `decouple=True weight_decay=0.01 betas=[0.9,0.999] d_coef=2 use_bias_correction=True safeguard_warmup=True`
optimizer_args = "weight_decay=0.1 betas=[0.9,0.99]" #@param {type:"string"}
optimizer_args = [a.strip() for a in optimizer_args.split(' ') if a]
#@markdown 如果選擇了 Dadapt 或 Prodigy 並勾選此框，則以下建議值將覆蓋任何先前的設定：<p>
#@markdown `unet_lr=0.75`, `text_encoder_lr=0.75`, `network_alpha=network_dim`
recommended_values_for_prodigy = True #@param {type:"boolean"}

if any(opt in optimizer.lower() for opt in ["dadapt", "prodigy"]):
  if recommended_values_for_prodigy:
    unet_lr = 0.75
    text_encoder_lr = 0.75
    network_alpha = network_dim

#@markdown 你可以在這裡寫下你的 Google Drive 中的路徑，以載入現有的 Lora 檔案，以繼續訓練。
#@markdown **警告** 這不是一個長時間的訓練階段。每個 epoch 都是從頭開始，並且可能會有更差的結果。
continue_from_lora = "" #@param {type:"string"}

import os

if continue_from_lora:
  if not continue_from_lora.startswith("/content/drive/MyDrive"):
    continue_from_lora = os.path.join("/content/drive/MyDrive", continue_from_lora)

  if not os.path.exists('/content/drive'):
    from google.colab import drive
    print("📂 連接到 Google Drive...")
    drive.mount('/content/drive')

  if not (continue_from_lora.endswith(".safetensors") and os.path.exists(continue_from_lora)):
    print(f"💥 錯誤：無法讀取既有 Lora 檔案，訓練將無法從此現有 Lora 繼續訓練。")

#@markdown ### ▶️ 準備好了
#@markdown 你現在可以執行此儲存格來開始訓練。祝你好運！<p>


# 👩‍💻 Cool code goes here

root_dir = "/content" if COLAB else "~/Loras"
deps_dir = os.path.join(root_dir, "deps")
repo_dir = os.path.join(root_dir, "kohya-trainer")

if "/Loras" in folder_structure:
  main_dir      = os.path.join(root_dir, "drive/MyDrive/Loras") if COLAB else root_dir
  log_folder    = os.path.join(main_dir, "_logs")
  config_folder = os.path.join(main_dir, project_name)
  images_folder = os.path.join(main_dir, project_name, "dataset")
  output_folder = os.path.join(main_dir, project_name, "output")
else:
  main_dir      = os.path.join(root_dir, "drive/MyDrive/lora_training") if COLAB else root_dir
  images_folder = os.path.join(main_dir, "datasets", project_name)
  output_folder = os.path.join(main_dir, "output", project_name)
  config_folder = os.path.join(main_dir, "config", project_name)
  log_folder    = os.path.join(main_dir, "log")

config_file = os.path.join(config_folder, "training_config.toml")
dataset_config_file = os.path.join(config_folder, "dataset_config.toml")
accelerate_config_file = os.path.join(repo_dir, "accelerate_config/config.yaml")

def install_dependencies():
  os.chdir(root_dir)
  !git clone {SOURCE} {repo_dir}
  os.chdir(repo_dir)
  if COMMIT:
    !git reset --hard {COMMIT}
  !wget https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/train_network_xl_wrapper.py -q -O train_network_xl_wrapper.py
  !wget https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/dracula.py -q -O dracula.py

  !apt -y update -qq
  !apt -y install aria2 -qq
  !pip install torch==2.2.1+cu121 accelerate==0.19.0 transformers==4.30.2 diffusers==0.18.2 bitsandbytes==0.40.0.post4 opencv-python==4.7.0.68 jax==0.4.23 jaxlib==0.4.23
  !pip install pytorch-lightning==1.9.0 voluptuous==0.13.1 toml==0.10.2 ftfy==6.1.1 einops==0.6.0 safetensors pygments
  !pip install huggingface-hub==0.20.3 invisible-watermark==0.2.0 open-clip-torch==2.20.0 dadaptation==3.1 prodigyopt==1.0 lion-pytorch==0.1.2
  !pip install -e .
  if cross_attention == "xformers":
    !pip install -q xformers==0.0.26.dev778

  # patch kohya for minor stuff
  if LOWRAM:
    !sed -i "s@cpu@cuda@" library/model_util.py
  if LOAD_TRUNCATED_IMAGES:
    !sed -i 's/from PIL import Image/from PIL import Image, ImageFile\nImageFile.LOAD_TRUNCATED_IMAGES=True/g' library/train_util.py # fix truncated jpegs error
  if BETTER_EPOCH_NAMES:
    !sed -i 's/{:06d}/{:02d}/g' library/train_util.py # make epoch names shorter
    !sed -i 's/"." + args.save_model_as)/"-{:02d}.".format(num_train_epochs) + args.save_model_as)/g' train_network.py # name of the last epoch will match the rest

  from accelerate.utils import write_basic_config
  if not os.path.exists(accelerate_config_file):
    write_basic_config(save_location=accelerate_config_file)

  os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
  os.environ["BITSANDBYTES_NOWELCOME"] = "1"
  os.environ["SAFETENSORS_FAST_GPU"] = "1"

def validate_dataset():
  global lr_warmup_steps, lr_warmup_ratio, caption_extension, keep_tokens
  supported_types = (".png", ".jpg", ".jpeg", ".webp", ".bmp")

  print("\n💿 檢查資料集...")
  if not project_name.strip() or any(c in project_name for c in " .()\"'\\/"):
    print("💥 錯誤：請選擇正確的專案名稱。")
    return

  if custom_dataset:
    try:
      datconf = toml.loads(custom_dataset)
      datasets = {d["image_dir"]: d["num_repeats"] for d in datconf["datasets"][0]["subsets"] if not (d["is_reg"] and d["is_reg"] == True)}
    except:
      print(f"💥 錯誤：你的自訂資料集結構錯誤，請確認資料集結構。")
      return
    reg = [d.get("image_dir") for d in datasets if d.get("is_reg", False)]
    datasets_dict = {d["image_dir"]: d["num_repeats"] for d in datasets}
    folders = datasets_dict.keys()
    files = [f for folder in folders for f in os.listdir(folder)]
    images_repeats = {folder: (len([f for f in os.listdir(folder) if f.lower().endswith(supported_types)]), datasets_dict[folder]) for folder in folders}
  else:
    reg = []
    folders = [images_folder]
    files = os.listdir(images_folder)
    images_repeats = {images_folder: (len([f for f in files if f.lower().endswith(supported_types)]), num_repeats)}

  for folder in folders:
    if not os.path.exists(folder):
      print(f"💥 錯誤：資料夾 {folder.replace('/content/drive/', '')} 不存在。")
      return
  for folder, (img, rep) in images_repeats.items():
    if not img:
      print(f"💥 錯誤：你的 {folder.replace('/content/drive/', '')} 資料夾沒有資料。")
      return
  for f in files:
    if not f.lower().endswith((".txt", ".npz")) and not f.lower().endswith(supported_types):
      print(f"💥 錯誤：錯誤的資料集結構 \"{f}\"，中斷執行。")
      return

  if not [txt for txt in files if txt.lower().endswith(".txt")]:
    caption_extension = ""

  if continue_from_lora and not (continue_from_lora.endswith(".safetensors") and os.path.exists(continue_from_lora)):
    print(f"💥 錯誤：錯誤的路徑或 Lora 不存在。範例：/content/drive/MyDrive/Loras/example.safetensors")
    return

  pre_steps_per_epoch = sum(img*rep for (img, rep) in images_repeats.values())
  steps_per_epoch = pre_steps_per_epoch/train_batch_size
  total_steps = max_train_steps or int(max_train_epochs*steps_per_epoch)
  estimated_epochs = int(total_steps/steps_per_epoch)
  lr_warmup_steps = int(total_steps*lr_warmup_ratio)

  for folder, (img, rep) in images_repeats.items():
    print("📁"+folder.replace("/content/drive/", "") + (" (Regularization)" if folder in reg else ""))
    print(f"📈 讀取 {img} 圖片，重複 {rep} 次，共有 {img*rep} 個步驟。")
  print(f"📉 訓練批次 {train_batch_size} 除以總步驟數量 {pre_steps_per_epoch}，得到每一輪次處理 {int(steps_per_epoch)} 個步驟。")
  if max_train_epochs:
    print(f"🔮 最大輪次 {max_train_epochs}，大約會有 {total_steps} 個總訓練步驟。")
  else:
    print(f"🔮 總訓練步驟 {total_steps}，並除以 {estimated_epochs} 個輪次。")

  if total_steps > 12000:
    print("💥 錯誤：你的總訓練步驟過高，可能會造成錯誤。中斷訓練。")
    return

  return True

def create_config():
  global dataset_config_file, config_file, model_file

  if override_config_file:
    config_file = override_config_file
    print(f"\n⭕ 使用自訂設定檔 {config_file}")
  else:
    config_dict = {
      "network_arguments": {
        "unet_lr": unet_lr,
        "text_encoder_lr": text_encoder_lr if not cache_text_encoder_outputs else 0,
        "network_dim": network_dim,
        "network_alpha": network_alpha,
        "network_module": network_module,
        "network_args": network_args,
        "network_train_unet_only": text_encoder_lr == 0 or cache_text_encoder_outputs,
        "network_weights": continue_from_lora if continue_from_lora else None
      },
      "optimizer_arguments": {
        "learning_rate": unet_lr,
        "lr_scheduler": lr_scheduler,
        "lr_scheduler_num_cycles": lr_scheduler_num_cycles if lr_scheduler == "cosine_with_restarts" else None,
        "lr_scheduler_power": lr_scheduler_power if lr_scheduler == "polynomial" else None,
        "lr_warmup_steps": lr_warmup_steps if lr_scheduler != "constant" else None,
        "optimizer_type": optimizer,
        "optimizer_args": optimizer_args if optimizer_args else None,
      },
      "training_arguments": {
        "pretrained_model_name_or_path": model_file,
        "vae": vae_file,
        "max_train_steps": max_train_steps,
        "max_train_epochs": max_train_epochs,
        "train_batch_size": train_batch_size,
        "seed": 42,
        "max_token_length": 225,
        "xformers": cross_attention == "xformers",
        "sdpa": cross_attention == "sdpa",
        "min_snr_gamma": min_snr_gamma if min_snr_gamma > 0 else None,
        "lowram": LOWRAM,
        "no_half_vae": True,
        "gradient_checkpointing": True,
        "gradient_accumulation_steps": 1,
        "max_data_loader_n_workers": 8,
        "persistent_data_loader_workers": True,
        "mixed_precision": mixed_precision,
        "full_bf16": mixed_precision == "bf16",
        "cache_latents": cache_latents,
        "cache_latents_to_disk": cache_latents_to_drive,
        "cache_text_encoder_outputs": cache_text_encoder_outputs,
        "min_timestep": 0,
        "max_timestep": 1000,
        "prior_loss_weight": prior_loss_weight,
      },
      "saving_arguments": {
        "save_precision": "fp16",
        "save_model_as": "safetensors",
        "save_every_n_epochs": save_every_n_epochs,
        "save_last_n_epochs": keep_only_last_n_epochs,
        "output_name": project_name,
        "output_dir": output_folder,
        "log_prefix": project_name,
        "logging_dir": log_folder,
      }
    }

    for key in config_dict:
      if isinstance(config_dict[key], dict):
        config_dict[key] = {k: v for k, v in config_dict[key].items() if v is not None}

    with open(config_file, "w") as f:
      f.write(toml.dumps(config_dict))
    print(f"\n📄 Config saved to {config_file}")

  if override_dataset_config_file:
    dataset_config_file = override_dataset_config_file
    print(f"⭕ Using custom dataset config file {dataset_config_file}")
  else:
    dataset_config_dict = {
      "general": {
        "resolution": resolution,
        "shuffle_caption": shuffle_caption and not cache_text_encoder_outputs,
        "keep_tokens": keep_tokens,
        "flip_aug": flip_aug,
        "caption_extension": caption_extension,
        "enable_bucket": True,
        "bucket_no_upscale": bucket_no_upscale,
        "bucket_reso_steps": 64,
        "min_bucket_reso": 256,
        "max_bucket_reso": 4096,
      },
      "datasets": toml.loads(custom_dataset)["datasets"] if custom_dataset else [
        {
          "subsets": [
            {
              "num_repeats": num_repeats,
              "image_dir": images_folder,
              "class_tokens": None if caption_extension else project_name
            }
          ]
        }
      ]
    }

    for key in dataset_config_dict:
      if isinstance(dataset_config_dict[key], dict):
        dataset_config_dict[key] = {k: v for k, v in dataset_config_dict[key].items() if v is not None}

    with open(dataset_config_file, "w") as f:
      f.write(toml.dumps(dataset_config_dict))
    print(f"\n📄 設定檔已儲存到 {config_file}")

def download_model():
  global old_model_url, model_url, model_file, vae_url, vae_file
  real_model_url = model_url.strip()

  if load_diffusers:
    if 'huggingface.co' in real_model_url:
        match = re.search(r'huggingface\.co/([^/]+)/([^/]+)', real_model_url)
        if match:
            username = match.group(1)
            model_name = match.group(2)
            model_file = f"{username}/{model_name}"
            fs = HfFileSystem()
            existing_folders = set(fs.ls(model_file, detail=False))
            necessary_folders = [ "scheduler", "text_encoder", "text_encoder_2", "tokenizer", "tokenizer_2", "unet", "vae" ]
            if all(f"{model_file}/{folder}" in existing_folders for folder in necessary_folders):
              print("🍃 Diffusers model identified.")
              return True
    print("🍃 無法讀取模型檔案。")

  if not model_file:
    if real_model_url.lower().endswith((".ckpt", ".safetensors")):
      model_file = f"/content{real_model_url[real_model_url.rfind('/'):]}"
    else:
      model_file = "/content/downloaded_model.safetensors"
      if os.path.exists(model_file):
        !rm "{model_file}"

  if m := re.search(r"(?:https?://)?(?:www\.)?huggingface\.co/[^/]+/[^/]+/blob", model_url):
    real_model_url = real_model_url.replace("blob", "resolve")
  elif m := re.search(r"(?:https?://)?(?:www\\.)?civitai\.com/models/([0-9]+)(/[A-Za-z0-9-_]+)?", model_url):
    if m.group(2):
      model_file = f"/content{m.group(2)}.safetensors"
    if m := re.search(r"modelVersionId=([0-9]+)", model_url):
      real_model_url = f"https://civitai.com/api/download/models/{m.group(1)}"
    else:
      raise ValueError("optional_custom_training_model_url contains a civitai link, but the link doesn't include a modelVersionId. You can also right click the download button to copy the direct download link.")

  !aria2c "{real_model_url}" --console-log-level=warn -c -s 16 -x 16 -k 10M -d / -o "{model_file}"

  if not os.path.exists(vae_file):
    !aria2c "{vae_url}" --console-log-level=warn -c -s 16 -x 16 -k 10M -d / -o "{vae_file}"

  if model_file.lower().endswith(".safetensors"):
    from safetensors.torch import load_file as load_safetensors
    try:
      test = load_safetensors(model_file)
      del test
    except:
      #if "HeaderTooLarge" in str(e):
      new_model_file = os.path.splitext(model_file)[0]+".ckpt"
      !mv "{model_file}" "{new_model_file}"
      model_file = new_model_file
      print(f"重新命名為 {os.path.splitext(model_file)[0]}.ckpt")

  if model_file.lower().endswith(".ckpt"):
    from torch import load as load_ckpt
    try:
      test = load_ckpt(model_file)
      del test
    except:
      return False

  return True

def main():
  global dependencies_installed

  if COLAB and not os.path.exists('/content/drive'):
    from google.colab import drive
    print("📂 Connecting to Google Drive...")
    drive.mount('/content/drive')

  for dir in (main_dir, deps_dir, repo_dir, log_folder, images_folder, output_folder, config_folder):
    os.makedirs(dir, exist_ok=True)

  if not validate_dataset():
    return

  if not dependencies_installed:
    print("\n🏭 安裝相依套件...\n")
    t0 = time()
    install_dependencies()
    t1 = time()
    dependencies_installed = True
    print(f"\n✅ 安裝完成，耗時 {int(t1-t0)} 秒。")
  else:
    print("\n✅ 相依套件已安裝。")

  if old_model_url != model_url or not model_file or not os.path.exists(model_file):
    print("\n🔄 下載模型...")
    if not download_model():
      print("\n💥 錯誤：您選擇的模型無效或已損壞，或無法下載。您可以使用 civitai 或 Huggingface 鏈接，或任何直接下載鏈接。")
      return
    print()
  else:
    print("\n🔄 模型已下載。\n")

  create_config()

  print("\n⭐ 啟動訓練...\n")
  os.chdir(repo_dir)

  !accelerate launch --quiet --config_file={accelerate_config_file} --num_cpu_threads_per_process=1 train_network_xl_wrapper.py --dataset_config={dataset_config_file} --config_file={config_file}

  if not get_ipython().__dict__['user_ns']['_exit_code']:
    display(Markdown("### ✅ 完成 [從 Google 雲端硬碟下載您的 Lora](https://drive.google.com/drive/my-drive)\n"
                     "### 會有幾個文件，你應該嘗試最新版本（旁邊數字最大的文件）"))

main()


## *️⃣ 擴充功能

在開始訓練之前，你可以執行以下的功能。

### 📚 複數資料夾的資料集
以下的樣版允許你在資料集中，定義複數的資料夾。你需要包含每個資料集的檔案路徑，且指定每個資料集的重複次數。你可以直接複製 `[[datasets.subsets]]` 區塊，簡單的增加你的資料集。

當你使用這個設定，在原本訓練中的重複次數的設定將會被忽略，且依據專案的資料集設定也會被忽略。

你可以加入 `ìs_reg = true` 將某一個資料集設定為正規化（regularization）資料。
你也可以設定各種不同的參數，例如 `keep_tokens`, `flip_aug` 等等。

In [1]:
custom_dataset = """
[[datasets]]

[[datasets.subsets]]
image_dir = "/content/drive/MyDrive/Loras/example/dataset/good_images"
num_repeats = 3
is_reg = false

[[datasets.subsets]]
image_dir = "/content/drive/MyDrive/Loras/example/dataset/normal_images"
num_repeats = 1
is_reg = false

[[datasets.subsets]]
image_dir = "/content/drive/MyDrive/Loras/example/dataset/reg_images"
num_repeats = 1
is_reg = true

"""

In [ ]:
custom_dataset = None

In [ ]:
#@markdown ### 📂 Unzip dataset
#@markdown It's much slower to upload individual files to your Drive, so you may want to upload a zip if you have your dataset in your computer.
zip = "/content/drive/MyDrive/my_dataset.zip" #@param {type:"string"}
extract_to = "/content/drive/MyDrive/Loras/example/dataset" #@param {type:"string"}

import os, zipfile

if not os.path.exists('/content/drive'):
  from google.colab import drive
  print("📂 Connecting to Google Drive...")
  drive.mount('/content/drive')

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip, 'r') as f:
  f.extractall(extract_to)

print("✅ 完成")


In [ ]:
#@markdown ### 🔢 Count datasets
#@markdown Google Drive makes it impossible to count the files in a folder, so this will show you the file counts in all folders and subfolders.
folder = "/content/drive/MyDrive/Loras" #@param {type:"string"}

import os
from google.colab import drive

if not os.path.exists('/content/drive'):
    print("📂 連接到 Google Drive...\n")
    drive.mount('/content/drive')

tree = {}
exclude = ("_logs", "/output")
for i, (root, dirs, files) in enumerate(os.walk(folder, topdown=True)):
  dirs[:] = [d for d in dirs if all(ex not in d for ex in exclude)]
  images = len([f for f in files if f.lower().endswith((".png", ".jpg", ".jpeg"))])
  captions = len([f for f in files if f.lower().endswith(".txt")])
  others = len(files) - images - captions
  path = root[folder.rfind("/")+1:]
  tree[path] = None if not images else f"{images:>4} images | {captions:>4} captions |"
  if tree[path] and others:
    tree[path] += f" {others:>4} other files"

pad = max(len(k) for k in tree)
print("\n".join(f"📁{k.ljust(pad)} | {v}" for k, v in tree.items() if v))


# 📈 繪製訓練結果
您可以在運行訓練器後執行此操作。除非您知道自己在做什麼，否則您不需要這個。
下面的第一個儲存格可能無法載入您的所有日誌。繼續嘗試第二個單元格，直到加載所有資料。

In [ ]:
%load_ext tensorboard
%tensorboard --logdir={log_folder}/

In [ ]:
from tensorboard import notebook
notebook.display(port=6006, height=800)